In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')
data.head()

In [ ]:
data = data.drop(columns=['sl_no','salary'])
data.head()

In [ ]:
y = data[['status']]

x = data.drop(columns=['status'])

In [ ]:
x.head()

In [ ]:
nom_cols = [0,5,7,10]
ord_cols = [2,4,8]

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,PowerTransformer

transf = make_column_transformer((OneHotEncoder(sparse=False),nom_cols),(OrdinalEncoder(),ord_cols),(PowerTransformer(method='box-cox'),[1,3,6,9,11]),remainder = "passthrough")

transf.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.3)

In [ ]:
data.shape

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
# from sklearn.model_selection import KFold

# kf = KFold(n_splits=3)

# for train,test in kf.split([0,1,2,3,4,5,6,7,8]):
#     print(train,test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)

model

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn import set_config

set_config(display="diagram")

pipe = make_pipeline(transf,model)

pipe

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pred = pipe.predict(x_test)

pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,precision_score

accuracy_score(pred,y_test)*100

In [ ]:
n_pred = pipe.predict(pd.DataFrame(np.array(['M',77.00,'Central',89.92,'Others','Science',74.00,'Sci&Tech','No',81.00,'Mkt&HR',62]).reshape(1,12)))
n_pred

In [ ]:
#Dumping

# import pickle

# pickle.dump(pipe, open('pipe.pkl','wb'))

#Loading back

# pipe = pickle.load(open('pipe.pkl','rb'))

In [ ]:
from sklearn.model_selection import StratifiedKFold,cross_val_score

s_fold = StratifiedKFold(n_splits=4,shuffle=True,random_state=3)

np.mean(cross_val_score(pipe, x,y, cv=s_fold, scoring="accuracy"))*100

In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(pipe, x_test, y_test, xticks_rotation='vertical',display_labels=['Placed','Not Placed'],cmap='Blues')

In [ ]:
import seaborn as sns

sns.countplot(y.status)

In [ ]:
y.status.value_counts()

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

r_under = RandomUnderSampler()

x_under, y_under = r_under.fit_resample(x,y)

sns.countplot(y_under.status)

In [ ]:
y_under.value_counts()

In [ ]:
from imblearn.over_sampling import RandomOverSampler

r_over = RandomOverSampler()

x_over, y_over = r_over.fit_resample(x,y)

sns.countplot(y_over.status)

In [ ]:
y_over.value_counts()

In [ ]:
x_over.head()

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

x_smote, y_smote = smote.fit_resample(transf.fit_transform(x),y)

sns.countplot(y_smote.status)

In [ ]:
y_smote.value_counts()

In [ ]:
from imblearn.pipeline import make_pipeline

pipe_smote = make_pipeline(transf,smote,model)

np.mean(cross_val_score(pipe_smote, x,y, cv=s_fold, scoring="accuracy"))*100

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

pipe_smote2 = make_pipeline(transf,lr)

np.mean(cross_val_score(pipe_smote, x,y, cv=s_fold, scoring="accuracy"))*100